In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import argparse
import logging
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.metrics import f1_score, classification_report
from torch.utils.data import DataLoader, TensorDataset
import sys
from tqdm import tqdm, trange
import os
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from torch.optim.lr_scheduler import CyclicLR


/home/student/miniforge3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import RobertaModel
import torch
import torch.nn as nn
from transformers.modeling_outputs import SequenceClassifierOutput

class RobertaWithMultiSampleDropoutTarget(nn.Module):
    def __init__(self, model_name, num_labels, dropout_rate=0.3, num_dropouts=5, use_multi_sample_dropout=True):
        super(RobertaWithMultiSampleDropoutTarget, self).__init__()
        self.roberta = RobertaModel.from_pretrained(model_name)
        self.use_multi_sample_dropout = use_multi_sample_dropout
        self.hidden_size = self.roberta.config.hidden_size

        if self.use_multi_sample_dropout:
            self.dropouts = nn.ModuleList([
                nn.Dropout(dropout_rate) for _ in range(num_dropouts)
            ])
        else:
            self.dropout = nn.Dropout(dropout_rate)

        self.classifier = nn.Linear(self.hidden_size, num_labels)

    def extract_entity_embeddings(self, input_ids, sequence_output, en_token_id, end_en_token_id):
        batch_size = input_ids.size(0)
        entity_representations = []

        for i in range(batch_size):
            input_id = input_ids[i]
            output = sequence_output[i]

            try:
                start = (input_id == en_token_id).nonzero(as_tuple=True)[0].item()
                end = (input_id == end_en_token_id).nonzero(as_tuple=True)[0].item()
            except IndexError:
                # если вдруг токены не найдены — fallback на [CLS]
                entity_representations.append(output[0])
                continue

            # вырезаем эмбеддинги между <en> и </en>
            entity_tokens = output[start + 1:end]
            if entity_tokens.shape[0] == 0:
                entity_representations.append(output[0])  # fallback на [CLS]
            else:
                entity_representations.append(entity_tokens.mean(dim=0))  # mean pooling

        return torch.stack(entity_representations)

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.roberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )
        sequence_output = outputs[0]  # [batch_size, seq_len, hidden_size]

        # --- 1. Находим позиции <en> и </en> ---
        start_token_id = start_token_ids = [tokenizer.convert_tokens_to_ids(f"<en:{tag}>") for tag in teg_class_entity]
        end_token_id = tokenizer.convert_tokens_to_ids("</en>")
    
        start_positions = torch.zeros_like(input_ids).bool()
        for token_id in start_token_ids:
            start_positions |= (input_ids == token_id)

        start_positions = start_positions.nonzero(as_tuple=False)
        end_positions = (input_ids == end_token_id).nonzero(as_tuple=False)

        # --- 2. Для каждого примера агрегируем hidden states между <en> и </en> ---
        pooled_output = []
        for batch_idx in range(input_ids.size(0)):
            start_pos = start_positions[start_positions[:, 0] == batch_idx][:, 1]
            end_pos = end_positions[end_positions[:, 0] == batch_idx][:, 1]
            if len(start_pos) > 0 and len(end_pos) > 0:
                s, e = start_pos[0].item() + 1, end_pos[0].item()  # между тегами
                if e > s:
                    token_embeds = sequence_output[batch_idx, s:e, :]  # [num_entity_tokens, hidden]
                    pooled = torch.mean(token_embeds, dim=0)  # Mean-pooling
                else:
                    pooled = sequence_output[batch_idx, 0, :]  # fallback to [CLS]
            else:
                pooled = sequence_output[batch_idx, 0, :]  # fallback to [CLS]

            pooled_output.append(pooled)

        pooled_output = torch.stack(pooled_output)  # [batch_size, hidden_size]

        if self.use_multi_sample_dropout:
            logits_list = [self.classifier(dropout(pooled_output)) for dropout in self.dropouts]
            avg_logits = torch.mean(torch.stack(logits_list), dim=0)
        else:
            avg_logits = self.classifier(self.dropout(pooled_output))

        return SequenceClassifierOutput(logits=avg_logits)


In [3]:
def save_metrics_best(epoch, all_labels, all_preds, result_path, backprop, loss):
    import os
    import pandas as pd
    from sklearn.metrics import classification_report

    metrics_file = os.path.join(result_path, "metrics_best.csv")
    report = classification_report(all_labels, all_preds, output_dict=True)

    # Создаем словарь с метриками
    metrics_data = {
        "epoch": epoch,
        "accuracy": report["accuracy"],
        "macro_precision": report["macro avg"]["precision"],
        "macro_recall": report["macro avg"]["recall"],
        "macro_f1": report["macro avg"]["f1-score"],
        "weighted_precision": report["weighted avg"]["precision"],
        "weighted_recall": report["weighted avg"]["recall"],
        "weighted_f1": report["weighted avg"]["f1-score"],
        "backprop": backprop,
        "loss": loss
    }

    # Добавляем метрики по каждому классу
    for label in sorted(report.keys()):
        if isinstance(report[label], dict):
            metrics_data[f"class_{label}_precision"] = report[label]["precision"]
            metrics_data[f"class_{label}_recall"] = report[label]["recall"]
            metrics_data[f"class_{label}_f1"] = report[label]["f1-score"]

    # Создаем DataFrame и сохраняем его в CSV (перезапись файла)
    metrics_df = pd.DataFrame([metrics_data])
    metrics_df.to_csv(metrics_file, mode='w', header=True, index=False)

In [4]:
def compute_ens_weights(train_labels, beta: float = 0.999):
    train_labels = np.array(train_labels)
    classes, counts = np.unique(train_labels, return_counts=True)
    effective_num = (1 - np.power(beta, counts)) / (1 - beta)
    weights = 1.0 / effective_num
    weights = weights / np.sum(weights) * len(classes)  # нормализация как в оригинальной статье
    return torch.tensor(weights, dtype=torch.float)

In [5]:
def get_entity_tags_from_files(*file_paths):
    all_tags = set()
    for path in file_paths:
        try:
            df = pd.read_csv(path, sep='\t')
            if "entity_tag" in df.columns:
                tags = df["entity_tag"].dropna().unique().tolist()
                all_tags.update(tags)
        except Exception as e:
            print(f"[Ошибка] Не удалось загрузить {path}: {e}")
    return sorted(list(all_tags))

In [6]:
def load_data(file_path, tokenizer, max_seq_len):
    try:
        df = pd.read_csv(file_path, sep='\t')  # Загружаем CSV (TSV)

        # Проверка нужных колонок
        required_columns = {
            "sentence", "entity", "label", "entity_tag",
            "entity_pos_start_rel", "entity_pos_end_rel"
        }
        if not required_columns.issubset(df.columns):
            raise ValueError(f"Ожидаемые колонки: {required_columns}, но в файле: {df.columns}")

        # Заменяем -1 на 2 (если есть)
        df["label"] = df["label"].replace(-1, 2)

        # Размечаем предложения с [ENTITY] токенами
        def mark_entity_inline(row):
            sent = row["sentence"]
            start = row["entity_pos_start_rel"]
            end = row["entity_pos_end_rel"]
            tag = row["entity_tag"]
            tag_token = f"<en:{tag}>"
            tag_token_close = f"</en>"
            return (
                sent[:start] +
                f"{tag_token} " + sent[start:end] + f" {tag_token_close}" +
                sent[end:]
            )

        df["input_text"] = df.apply(mark_entity_inline, axis=1)
        texts = df["input_text"].tolist()
        labels = torch.tensor(df["label"].astype(int).tolist(), dtype=torch.long)

        # Токенизируем
        encodings = tokenizer(texts, padding=True, truncation=True,
                              max_length=max_seq_len, return_tensors='pt')
        dataset = TensorDataset(encodings['input_ids'], encodings['attention_mask'], labels)
        return dataset

    except FileNotFoundError:
        logging.error(f"Файл {file_path} не найден!")
        return None

In [7]:

def save_metrics(epoch, all_labels, all_preds, result_path, backprop, loss):
    metrics_file = os.path.join(result_path, "metrics_class_weights.csv")
    report = classification_report(all_labels, all_preds, output_dict=True)
    
    # Создаем DataFrame для метрик
    metrics_data = {
        "epoch": epoch,
        "accuracy": report["accuracy"],
        "macro_precision": report["macro avg"]["precision"],
        "macro_recall": report["macro avg"]["recall"],
        "macro_f1": report["macro avg"]["f1-score"],
        "weighted_precision": report["weighted avg"]["precision"],
        "weighted_recall": report["weighted avg"]["recall"],
        "weighted_f1": report["weighted avg"]["f1-score"],
        "backprop": backprop , # (Добавлено)
        "loss": loss
    }
    
    # Добавляем метрики для каждого класса
    for label in sorted(report.keys()):
        if isinstance(report[label], dict):  # Пропускаем 'accuracy', так как это float
            metrics_data[f"class_{label}_precision"] = report[label]["precision"]
            metrics_data[f"class_{label}_recall"] = report[label]["recall"]
            metrics_data[f"class_{label}_f1"] = report[label]["f1-score"]
    
    # Создаем DataFrame и сохраняем в CSV
    metrics_df = pd.DataFrame([metrics_data])
    metrics_df.to_csv(metrics_file, mode='a', header=not os.path.exists(metrics_file), index=False)

def save_confusion_matrix(epoch, y_true, y_pred, result_path,backprop):
    cm = confusion_matrix(y_true, y_pred)
    cm_df = pd.DataFrame(cm, columns=["Pred_0", "Pred_1", "Pred_2"], index=["True_0", "True_1", "True_2"])
    cm_file = os.path.join(result_path, f"confusion_matrix_class_weights_epoch_{epoch}_backprop_{backprop}.csv")
    cm_df.to_csv(cm_file)



In [8]:
from sklearn.metrics import classification_report
import pandas as pd

def castom_classification_report(all_labels, all_preds):
    report = classification_report(all_labels, all_preds, output_dict=True)
    data = []
    for label, metrics in report.items():
        if isinstance(metrics, dict):  # Для всех метрик, кроме accuracy
            row = {'label': label}
            for metric, value in metrics.items():
                row[metric] = round(value * 100, 3) if metric != 'support' else value
            data.append(row)
        else:  # Для accuracy
            data.append({'label': 'accuracy', 'precision': round(metrics * 100, 3), 'recall': None, 'f1-score': None, 'support': None})

    # Добавляем macro F1 для классов 1 и 2
    if '1' in report and '2' in report:
        f1_1 = report['1']['f1-score']
        f1_2 = report['2']['f1-score']
        f1_macro_1_2 = (f1_1 + f1_2) / 2
        data.append({
            'label': 'avg f1 (class 1&2)',
            'precision': None,
            'recall': None,
            'f1-score': round(f1_macro_1_2 * 100, 3),
            'support': None
        })
    
    # Создаем DataFrame
    df = pd.DataFrame(data)
    df.index = [''] * len(df)

    # Выводим таблицу
    print(df.to_string(index=False))

In [9]:
def add_token(flag = False):
    # Формируем список специальных токенов
    if not flag: return None
    #entity_tags = get_entity_tags_from_files(args.train_data, args.validation_data, args.eval_data)
    special_tokens = {
        "additional_special_tokens":
            [f"<en:{tag}>" for tag in teg_class_entity] +
            ["<en>", "</en>"]
    }

    print("Добавленные специальные токены:")
    for token in special_tokens["additional_special_tokens"]:
        print(token)
    return special_tokens

def save_contrel_date(tokenizer, train_dataset):
    special_tokens = add_token(flag = True)
    if special_tokens != None: 
        tokenizer.add_special_tokens(special_tokens)
    tokenizer.save_pretrained(os.path.join(args.result, "tokenizer"))
    print(tokenizer.special_tokens_map)
    print(tokenizer.additional_special_tokens)

    
    file_control_text = os.path.join(args.result, "use_market_text.txt")
    input_ids, attention_mask, label = train_dataset[0]
    decoded_text_token = tokenizer.decode(input_ids, skip_special_tokens=False)
    decoded_text = tokenizer.decode(input_ids, skip_special_tokens=True)
    with open(file_control_text, "w") as f:
        f.write(f"Text token:\n{decoded_text_token}\n\n")
        f.write(f"Text:\n{decoded_text}\n\n")
    print(f"Text token:\n{decoded_text_token}\n\n")
    print(f"Text:\n{decoded_text}\n\n")

In [10]:
teg_class_entity = ["COUNTRY", "NATIONALITY", "ORGANIZATION", "PERSON", "PROFESSION"]
lr_m = 1e-6
# lr = 1e-6 началось обучение, уменьшение TrainError
# добавить волидацию по шагам в течение эпохи обучения, а не каджые N епох
# Добавить сохранение ошибки в файлы для построения графиков
# проверить методы передачи сущьности (параетром, +тип, или в тексте выделяя тегами).
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_name", type=str, default="./../ruRoberta-large/")
    #parser.add_argument("--model_name", type=str, default="sberbank-ai/ruRoberta-large", help="Имя модели")
    parser.add_argument("--max_seq_len", type=int, default=512) #128
    parser.add_argument("--batch_size", type=int, default=16)
    parser.add_argument("--epochs", type=int, default=10)
    parser.add_argument("--lr", type=float, default=lr_m)
    parser.add_argument("--init_checkpoint", type=str, default=None)
    parser.add_argument("--train_data", type=str, required=False, default="./../data/train_data.csv")
    parser.add_argument("--validation_data", type=str, required=False, default="./../data/validation.csv")
    parser.add_argument("--eval_data", type=str, required=False, default="./../data/test.csv")
    parser.add_argument("--result", type=str, default=f"./result_lr_{lr_m}_ENS_Teg_34_multi_dropout_V3_Token_Target/")
    # Игнорируем аргументы Jupyter
    args, unknown = parser.parse_known_args()
    os.makedirs(args.result, exist_ok=True)
    return args


if __name__ == "__main__":
    args = parse_args()
    print(args)  # Проверяем аргументы

    base_lr = args.lr
    min_lr = base_lr * (1/4)
    step_size_up = 400 #400
    best_avg_f1 = 0.0
    
    best_model_path = os.path.join(args.result, "best_model.pth")

    SEED = 42
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    
    logging.basicConfig(level=logging.INFO)

    tokenizer = RobertaTokenizer.from_pretrained(args.model_name)
    #model = RobertaForSequenceClassification.from_pretrained(args.model_name, num_labels=3)
    model = RobertaWithMultiSampleDropoutTarget(model_name='./../ruRoberta-large', num_labels=3, use_multi_sample_dropout=True )
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    train_dataset = load_data(args.train_data, tokenizer, args.max_seq_len)
    validation_data = load_data(args.validation_data, tokenizer, args.max_seq_len)
    eval_dataset = load_data(args.eval_data, tokenizer, args.max_seq_len)

    if train_dataset is None or eval_dataset is None or validation_data is None:
        sys.exit(f"Ошибка загрузки данных: убедитесь, что файлы {args.train_data}, {args.validation_data} и {args.eval_data} существуют и содержат нужные колонки.")

    #сохранение текста и токенов, с дабовлением собственных
    save_contrel_date(tokenizer,train_dataset)
    model.roberta.resize_token_embeddings(len(tokenizer))
    
    embedding_layer = model.roberta.embeddings.word_embeddings
    print(embedding_layer.weight[tokenizer.convert_tokens_to_ids("<en>")])
    
    # Получаем метки классов из тренировочного датасета
    train_labels = [label.item() for _, _, label in train_dataset]

    # Вычисляем веса классов
    class_weights = compute_ens_weights(train_labels, beta=0.999)

    # Вывод весов классов
    print(f"class_weights = {class_weights}")
    
    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    validation_loader = DataLoader(validation_data, batch_size=args.batch_size)
    eval_loader = DataLoader(eval_dataset, batch_size=args.batch_size)
    
    optimizer = optim.AdamW(model.parameters(), lr=args.lr)
    
    #scheduler = CyclicLR(optimizer, base_lr=min_lr, max_lr = base_lr, step_size_up = step_size_up, mode="triangular2",cycle_momentum=False )

    loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(device))
    
    def train():
        model.train()
        batches_per_validation = (len(train_loader) // 2)+2
        batches_per_test = (len(train_loader) // 5)+2
        
        for epoch in range(args.epochs):
            print(f"Epoch: {epoch}/{args.epochs}")
            total_loss = 0
            for i,batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1} - Batches")):
                input_ids, attention_mask, labels = [x.to(device) for x in batch]
                optimizer.zero_grad()
                outputs = model(input_ids, attention_mask=attention_mask)
                loss = loss_fct(outputs.logits, labels)
                loss.backward()
                optimizer.step()
                #scheduler.step()
                
                total_loss += loss.item()
                if(i % batches_per_validation == 0 and i >50):
                    evaluate(epoch, backprop="Validation", vall_train = True)
                if(i % batches_per_test == 0 and i >50):
                    evaluate(epoch, backprop="Test", flag_print = True)
                
            logging.info(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}, Step: {len(train_loader)}")
            # Валидация с обратным распространением ошибки каждые 2 эпохи
            evaluate(epoch , backprop="Train", flag_print = True)
            evaluate(epoch , backprop="Test", flag_print = True)
            evaluate(epoch , backprop="Validation", flag_print = True, vall_train = True)
            
            
    
    def evaluate(epoch = None, backprop = "None", flag_print = False, vall_train = False):
        global best_avg_f1
        model.eval()
        all_preds, all_labels = [], []
        total_loss = 0
        print(f"evaluate, backprop: {backprop}")
        loader = validation_loader
        if backprop == "Validation":
            loader = validation_loader
        elif backprop == "Test":
            loader = eval_loader
        elif backprop == "Train":
            loader = train_loader
        
        else: loader = eval_loader
        with torch.no_grad() if not backprop == "Validation" else torch.enable_grad():  # Включаем градиенты для обучения на валидации
            for batch in loader:
            #for batch in loader:
                input_ids, attention_mask, labels = [x.to(device) for x in batch]
                outputs = model(input_ids, attention_mask=attention_mask)
                loss = loss_fct(outputs.logits, labels)
                
                
                
                if backprop == "Validation" and vall_train:
                    #уменьшая тк обучающие данные удвоились
                    loss = loss
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    #scheduler.step()
                    loss = loss

                preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
                all_preds.extend(preds)
                all_labels.extend(labels.cpu().numpy())
                total_loss += loss.item()
                
        avg_loss = total_loss / len(loader)
        f1 = f1_score(all_labels, all_preds, average='macro')
        f1_pn = f1_score(all_labels, all_preds, labels=[1, 2], average='macro')
        avg_f1 = (f1 + f1_pn) / 2

        if(flag_print):
            print()
            print("--"*20)
            print(f"Result evaluate in {backprop}")
            #logging.info(f"{backprop} Loss: {avg_loss:.4f}")
            print(f"Loss: {avg_loss:.4f}")
            print(f"F1-macro: {f1:.4f}")
            print(f"F1-pn: {f1_pn:.4f}")
            #print(classification_report(all_labels, all_preds))
            castom_classification_report(all_labels, all_preds)

        # 💾 Сохраняем модель только при улучшении средней метрики
        if (avg_f1 > best_avg_f1) and backprop == "Test":
            best_avg_f1 = avg_f1
            torch.save(model.state_dict(), best_model_path)
            
            print(f"[Checkpoint] 🎯 Новый лучший средний F1: {best_avg_f1:.4f}")
            print(f"[Checkpoint] 💾 Модель сохранена: {best_model_path}")
            # 🔸 Сохраняем значение лучшего F1 в файл
            best_score_path = os.path.join(args.result, "best_score.txt")
            save_metrics_best(epoch, all_labels, all_preds, args.result, backprop, avg_loss)
            
            save_confusion_matrix(epoch, all_labels, all_preds, args.result, backprop)
            with open(best_score_path, "w") as f:
                f.write(f"Epoch: {epoch}\n")
                f.write(f"Loss: {avg_loss:.4f}\n")
                f.write(f"F1-pn0: {f1:.4f}\n")
                f.write(f"F1-pn: {f1_pn:.4f}\n")
            print(f"Loss: {avg_loss:.4f}")
            print(f"F1-macro: {f1:.4f}")
            print(f"F1-pn: {f1_pn:.4f}")


        if epoch is not None:
            save_metrics(epoch, all_labels, all_preds, args.result, backprop, avg_loss)
            
        
    if args.init_checkpoint:
        model.load_state_dict(torch.load(args.init_checkpoint, map_location=device))

    train()

Namespace(model_name='./../ruRoberta-large/', max_seq_len=512, batch_size=16, epochs=10, lr=1e-06, init_checkpoint=None, train_data='./../data/train_data.csv', validation_data='./../data/validation.csv', eval_data='./../data/test.csv', result='./result_lr_1e-06_ENS_Teg_34_multi_dropout_V3_Token_Target/')


Some weights of RobertaModel were not initialized from the model checkpoint at ./../ruRoberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Добавленные специальные токены:
<en:COUNTRY>
<en:NATIONALITY>
<en:ORGANIZATION>
<en:PERSON>
<en:PROFESSION>
<en>
</en>
{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>', 'additional_special_tokens': ['<en:COUNTRY>', '<en:NATIONALITY>', '<en:ORGANIZATION>', '<en:PERSON>', '<en:PROFESSION>', '<en>', '</en>']}
['<en:COUNTRY>', '<en:NATIONALITY>', '<en:ORGANIZATION>', '<en:PERSON>', '<en:PROFESSION>', '<en>', '</en>']
Text token:
<s>Джеймс «Бадди» Макгирт (James (Buddy) McGirt, тренер Дадашева упрашивал дагестанского <en:PROFESSION> спортсмена </en> остановить бой, но тот хотел продолжать.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

Epoch 1 - Batches:  20%|██        | 85/415 [00:48<03:06,  1.77it/s]

evaluate, backprop: Test

----------------------------------------
Result evaluate in Test
Loss: 0.7943
F1-macro: 0.2838
F1-pn: 0.0088
             label  precision  recall  f1-score  support
                 0     71.645  99.675    83.367    616.0
                 1     50.000   0.901     1.770    111.0
                 2      0.000   0.000     0.000    133.0
          accuracy     71.512     NaN       NaN      NaN
         macro avg     40.548  33.525    28.379    860.0
      weighted avg     57.771  71.512    59.943    860.0
avg f1 (class 1&2)        NaN     NaN     0.885      NaN


...

Epoch 9 - Batches:  82%|████████▏ | 341/415 [04:29<03:11,  2.59s/it]


----------------------------------------
Result evaluate in Test
Loss: 0.6714
F1-macro: 0.7402
F1-pn: 0.6645
             label  precision  recall  f1-score  support
                 0     90.333  87.987    89.145    616.0
                 1     64.286  64.865    64.574    111.0
                 2     64.865  72.180    68.327    133.0
          accuracy     82.558     NaN       NaN      NaN
         macro avg     73.161  75.011    74.015    860.0
      weighted avg     83.033  82.558    82.754    860.0
avg f1 (class 1&2)        NaN     NaN    66.451      NaN


Epoch 9 - Batches: 100%|██████████| 415/415 [05:10<00:00,  1.33it/s]
INFO:root:Epoch 9, Loss: 0.03388175776239529, Step: 415


evaluate, backprop: Train


KeyboardInterrupt: 

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

# Загружаем токенизатор и модель
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Добавим специальные токены
special_tokens = {"additional_special_tokens": ["<en>", "</en>"]}
tokenizer.add_special_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))

# Пример текста
text = "This is <en> Apple  company </en>."

# Токенизация
encoding = tokenizer(text, return_tensors='pt')
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

# Пропускаем через модель
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    sequence_output = outputs.last_hidden_state

# Получаем ID спецтокенов
en_token_id = tokenizer.convert_tokens_to_ids("<en>")
end_en_token_id = tokenizer.convert_tokens_to_ids("</en>")

# Извлекаем эмбеддинги сущности
input_id = input_ids[0]
output = sequence_output[0]

# Ищем индексы <en> и </en>
start = (input_id == en_token_id).nonzero(as_tuple=True)[0].item()
end = (input_id == end_en_token_id).nonzero(as_tuple=True)[0].item()

# Эмбеддинги между <en> и </en>
entity_tokens = output[start + 1:end]

print(f"Токены сущности: {tokenizer.convert_ids_to_tokens(input_id[start+1:end])}")
print(f"Форма эмбеддингов: {entity_tokens.shape}")


# Среднее по токенам сущности
entity_embedding = entity_tokens.mean(dim=0)
print(f"Размерность итогового эмбеддинга сущности: {entity_embedding.shape}")
